# Build random forest
spark RF mllib
https://spark.apache.org/docs/2.2.0/mllib-ensembles.html#random-forests
https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/    data download link 
https://stackoverflow.com/questions/30298523/spark-create-rdd-of-label-features-pairs-from-csv-file    how to prepare data

tune RF for better performance guide
https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/

what is random forest ?
https://www.coursera.org/learn/python-machine-learning/lecture/lF9QN/random-forests
Typically, for a classification problem with p features, √p (rounded down) features are used in each split.[3]:592 For regression problems the inventors recommend p/3 (rounded down) with a minimum node size of 5 as the default. wiki
training set bagging + feature bagging

implement random forest from scratch
https://www.youtube.com/watch?v=QHOazyP-YlM


off topic
Algorithms Specialization
Learn To Think Like A Computer Scientist. Master the fundamentals of the design and analysis of algorithms.
https://www.coursera.org/specializations/algorithms?action=enroll

Example German credit
https://mapr.com/blog/predicting-loan-credit-risk-using-apache-spark-machine-learning-random-forests/
https://archive.ics.uci.edu/ml/datasets/Statlog+(German+Credit+Data)     data download link

A very good comparison using various model to determin diabetes, also explain the features in the model
https://towardsdatascience.com/machine-learning-for-diabetes-562dd7df4d42


Uncompressing a .Z file
Files ending in .Z have been compressed using the compress command. Compressed files can be restored to their original form using the uncompress command. Type:
```
        uncompress filename.tar.Z 
```
This replaces filename.tar.Z with filename.tar

Unzipping a .gz file
Files ending in .gz have been compressed using the gzip command. Zipped files can be restored to their original form using the gunzip command. Type:
```
         gunzip filename.tar.gz 
```
This replaces filename.tar.gz with filename.tar

Extracting tar files
The tar command is used for packing several files or even a whole directory into a single tarfile.
To unpack a tar file, type:
```
         tar xvf filename.tar 
```
Note: Always move to an empty directory before unpacking a tarfile. 
When you unpack a tarfile its contents are written to the current directory. Any file or directory with the same name as the contents of the tarfile will be overwritten!

A quick method
Alternatively, you can unpack a compressed/zipped tarfile using the zcat or gzcat command. (gzcat is a local alias for the GNU zcat command). This leaves the compressed .Z or .gz tarfile intact. For example:
```
        zcat filename.tar.Z  | tar xvf - 
# or
       gzcat filename.tar.gz | tar xvf -
```
The zcat command recreates the uncompressed tarfile, which is then piped into the tar command to extract the files.


* diabetes-data.tar.Z contains the distribution for 70 sets of data recorded
on diabetes patients (several weeks' to months' worth of glucose, insulin,
and lifestyle data per patient + a description of the problem domain).
Archived using tar and compressed.

The Code field is deciphered as follows:

33 = Regular insulin dose
34 = NPH insulin dose
35 = UltraLente insulin dose

48 = Unspecified blood glucose measurement
57 = Unspecified blood glucose measurement
58 = Pre-breakfast blood glucose measurement
59 = Post-breakfast blood glucose measurement
60 = Pre-lunch blood glucose measurement
61 = Post-lunch blood glucose measurement
62 = Pre-supper blood glucose measurement
63 = Post-supper blood glucose measurement
64 = Pre-snack blood glucose measurement

65 = Hypoglycemic symptoms

66 = Typical meal ingestion
67 = More-than-usual meal ingestion
68 = Less-than-usual meal ingestion

69 = Typical exercise activity
70 = More-than-usual exercise activity
71 = Less-than-usual exercise activity

72 = Unspecified special event


https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/diabetes
the data is already cleaned for machine learning, but do not have feature names
```
-1  1:6.000000 2:148.000000 3:72.000000 4:35.000000 5:0.000000 6:33.599998 7:0.627000 8:50.000000
```

https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/diabetes.csv
a user has identify how the correspondance of data
```
Pregnancies	Glucose	BloodPressure	SkinThickness	Insulin	BMI	DiabetesPedigreeFunction	Age	Outcome
6	148	72	35	0	33.6	0.627	50	1
```

In [70]:
import findspark
findspark.init()

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

import pyspark
sc = pyspark.SparkContext.getOrCreate()

# model output dir setup
import os
import shutil
outputDir = '/predix/target/tmp/myRandomForestClassificationModel/'
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)

In [71]:
#
# Option A: load a csv with headers and convert to LIBSVM format for ML RF
#
    
# Load and parse the data file into an RDD
csvRDD = sc.textFile("/media/sf_Windows_Share/diabetes.csv")
print('csvRDD:{}\n{}\n'.format(csvRDD.count(), csvRDD.take(6)))

# To find the headers
# first() method returns a string not a Rdd. 
header = csvRDD.first()  # or csvRDD.take(1)
print(header)



csvRDD:769
['Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome', '6,148,72,35,0,33.6,0.627,50,1', '1,85,66,29,0,26.6,0.351,31,0', '8,183,64,0,0,23.3,0.672,32,1', '1,89,66,23,94,28.1,0.167,21,0', '0,137,40,35,168,43.1,2.288,33,1']

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [72]:
#
# Option A: option 1.1)
#

# remove hearder
csvRDD = csvRDD.filter(lambda line: line != header)
csvRDD.take(2)


['6,148,72,35,0,33.6,0.627,50,1', '1,85,66,29,0,26.6,0.351,31,0']

In [57]:
#
# Option A: option 1.2)
#

# convert header to rdd by using parallelize.
header = sc.parallelize([header])
print('sv.parallelize([header]):{}\n{}\n'.format(header.count(), header.take(6)))
# To remove the header subtract Rdd only, Rdd seems has no order
csvRDD = csvRDD.subtract(header)
csvRDD.take(6)


sv.parallelize([header]):1
['Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome']



['0,137,40,35,168,43.1,2.288,33,1',
 '10,168,74,0,0,38,0.537,34,1',
 '5,166,72,19,175,25.8,0.587,51,1',
 '0,118,84,47,230,45.8,0.551,31,1',
 '1,115,70,30,96,34.6,0.529,32,1',
 '7,147,76,0,0,39.4,0.257,43,1']

In [73]:
#
# Option A: load a csv with headers and convert to LIBSVM format for ML RF
#

# http://www.techpoweredmath.com/spark-dataframes-mllib-tutorial/#.WlFRNlunE2w

from pyspark.mllib.regression import LabeledPoint

def createLabeledPoint(line):
    features = line.split(',')
    return LabeledPoint(features[8], features[0:8])  # yield can't be used here, only return work
# https://spark.apache.org/docs/2.1.0/mllib-data-types.html#labeled-point

    
data = csvRDD.map(createLabeledPoint)
print(data.take(2))


# //To create a RDD of (label, features) pairs
# val parsedData = data.map { line =>
#     val parts = line.split(',')
#     LabeledPoint(parts(0).toDouble, Vectors.dense(parts(1).split(' ').map(_.toDouble)))
#     }.cache()


[LabeledPoint(1.0, [6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0]), LabeledPoint(0.0, [1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0])]


In [9]:
#
# Option B: Directly use data with LIBSVM format. 
#

# Load and parse the data file into an RDD of LabeledPoint.
# https://spark.apache.org/docs/2.2.0/mllib-data-types.html#labeled-point
data = MLUtils.loadLibSVMFile(sc, '/media/sf_Windows_Share/diabetesLIBSVM.csv')

# downloaeded LIBSVMFILE scale is [-1,+1] which won't work, change [-1, 0]
# LibSVMFile example: 
# 0   1:6.000000 2:148.000000 3:72.000000 4:35.000000 5:0.000000 6:33.599998 7:0.627000 8:50.000000
# +1  1:1.000000 2:85.000000 3:66.000000 4:29.000000 5:0.000000 6:26.600000 7:0.351000 8:31.000000
# 0   1:8.000000 2:183.000000 3:64.000000 4:0.000000 5:0.000000 6:23.299999 7:0.672000 8:32.000000
# +1  1:1.000000 2:89.000000 3:66.000000 4:23.000000 5:94.000000 6:28.100000 7:0.167000 8:21.000000
                                
print(data.take(2))

# loading data into a structure: first column is label, rest are indexed features:
# [LabeledPoint(0.0, (8,[0,1,2,3,4,5,6,7],[6.0,148.0,72.0,35.0,0.0,33.599998,0.627,50.0])),
#  LabeledPoint(1.0, (8,[0,1,2,3,4,5,6,7],[1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0]))]



[LabeledPoint(0.0, (8,[0,1,2,3,4,5,6,7],[6.0,148.0,72.0,35.0,0.0,33.599998,0.627,50.0])), LabeledPoint(1.0, (8,[0,1,2,3,4,5,6,7],[1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0]))]


In [59]:
# Knowledge: LabeledPoint, features, label
x = LabeledPoint(1.0, [1.0, 0.0, 3.0])
print(x.features)
print(x.label)

# Knowledge: rdd1.zip(rdd2)
# https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd-pysrc.html#RDD.zip
# https://www.safaribooksonline.com/library/view/learning-spark/9781449359034/ch04.html
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))
x.zip(y).collect()  # [(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]



[1.0,0.0,3.0]
1.0


[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

In [74]:
# run RF model
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
# Empty categoricalFeaturesInfo indicates all features are continuous.
# In pyspark you would need a syntax like this n:m , where n is the column of categorical Features(such as "pass", "BMW",
# which algo can't use, needs converting to a number, in some algo it must a continuous number)
# m is the number of categories minus 1, and you can have multiple columns with 
# categorical variables each seperated with a comma.
# categoricalFeaturesInfo={0:12,2:10,.....,10:2}

#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32, seed=1)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

# Save and load model
model.save(sc, outputDir)
sameModel = RandomForestModel.load(sc, outputDir)

# sc.stop()



Test Error = 0.26540284360189575
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 5 <= 29.8)
     If (feature 7 <= 31.0)
      If (feature 1 <= 105.0)
       Predict: 0.0
      Else (feature 1 > 105.0)
       If (feature 0 <= 7.0)
        Predict: 0.0
       Else (feature 0 > 7.0)
        Predict: 1.0
     Else (feature 7 > 31.0)
      If (feature 1 <= 155.0)
       If (feature 6 <= 0.839)
        Predict: 0.0
       Else (feature 6 > 0.839)
        Predict: 1.0
      Else (feature 1 > 155.0)
       If (feature 7 <= 62.0)
        Predict: 1.0
       Else (feature 7 > 62.0)
        Predict: 0.0
    Else (feature 5 > 29.8)
     If (feature 7 <= 30.0)
      If (feature 5 <= 42.4)
       If (feature 4 <= 392.0)
        Predict: 0.0
       Else (feature 4 > 392.0)
        Predict: 1.0
      Else (feature 5 > 42.4)
       If (feature 1 <= 94.0)
        Predict: 0.0
       Else (feature 1 > 94.0)
        Predict: 1.0
     Else (feature 

In [75]:
n = 9

print('\ntestData:')
print(testData.take(n))

print('\npredictions for testData:')
print(predictions.take(n))

print('\nlabelsAndPredictions:')
print(labelsAndPredictions.take(n))

print('\nlabelsAndPredictions - get only wrong prediction:')
print(labelsAndPredictions.filter(lambda lp: lp[0] != lp[1]).take(n))

print('\nuse saved model to predict testData:') 
sameModel.predict(testData.map(lambda x: x.features)).take(n)




testData:
[LabeledPoint(1.0, [6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0]), LabeledPoint(1.0, [10.0,168.0,74.0,0.0,0.0,38.0,0.537,34.0]), LabeledPoint(1.0, [7.0,100.0,0.0,0.0,0.0,30.0,0.484,32.0]), LabeledPoint(1.0, [1.0,115.0,70.0,30.0,96.0,34.6,0.529,32.0]), LabeledPoint(1.0, [9.0,119.0,80.0,35.0,0.0,29.0,0.263,29.0]), LabeledPoint(1.0, [10.0,125.0,70.0,26.0,115.0,31.1,0.205,41.0]), LabeledPoint(0.0, [1.0,97.0,66.0,15.0,140.0,23.2,0.487,22.0]), LabeledPoint(0.0, [5.0,117.0,92.0,0.0,0.0,34.1,0.337,38.0]), LabeledPoint(0.0, [3.0,88.0,58.0,11.0,54.0,24.8,0.267,22.0])]

predictions for testData:
[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]

labelsAndPredictions:
[(1.0, 1.0), (1.0, 1.0), (1.0, 0.0), (1.0, 1.0), (1.0, 0.0), (1.0, 1.0), (0.0, 0.0), (0.0, 1.0), (0.0, 0.0)]

labelsAndPredictions - get only wrong prediction:
[(1.0, 0.0), (1.0, 0.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 0.0)]

use saved model to predict testData:


[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]

In [77]:
# https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html#binary-classification

from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Compute raw scores on the test set
predictionAndLabels = predictions.zip(testData.map(lambda lp: lp.label))

# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)


# http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.98.4362&rep=rep1&type=pdf



Area under PR = 0.6588483503114162
Area under ROC = 0.6814048760991207


http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
http://www.chioka.in/differences-between-roc-auc-and-pr-auc/

**Precision** is defined as the number of true positives over the number of true positives plus the number of false positives.
In simple words, the Precesion is a ratio presents: 
    how many positive prediction are true positive / how many positive predictions given
    (the correctness of the positive prediction)

**Recall** is defined as the number of true positives over the number of true positives plus the number of false negatives.
In simple words, the Recall is a ration presents:
    how many positive prediction are true positive / how many positive really exist
    (the ability of discoverying positive)
    
the harmonic mean of precision and recall ==> PR

Note that the precision may not decrease with recall. The definition of precision shows that lowering the threshold of a classifier may increase the denominator, by increasing the number of results returned. If the threshold was previously set too high, the new results may all be true positives, which will increase precision. If the previous threshold was about right or too low, further lowering the threshold will introduce false positives, decreasing precision.

Recall is defined as , where TP+FN does not depend on the classifier threshold. This means that lowering the classifier threshold may increase recall, by increasing the number of true positive results. It is also possible that lowering the threshold may leave recall unchanged, while the precision fluctuates.

scikit trun RF blackbox more clearly:
http://blog.datadive.net/interpreting-random-forests/
http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/

House Data
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names
Attribute Information:

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per 10,000
    11. PTRATIO  pupil-teacher ratio by town
    12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks 
                 by town
    13. LSTAT    % lower status of the population
    14. MEDV     Median value of owner-occupied homes in $1000's

https://mapr.com/blog/predicting-loan-credit-risk-using-apache-spark-machine-learning-random-forests/


Using an ML pipeline
We will next train the model using a pipeline, which can give better results. A pipeline provides a simple way to try out different combinations of parameters, using a process called grid search, where you set up the parameters to test, and MLLib will test all the combinations. Pipelines make it easy to tune an entire model building workflow at once, rather than tuning each element in the Pipeline separately.


https://spark.apache.org/docs/2.2.0/ml-tuning.html

